## Twitter Scraper
Given a csv of twitter users, login to twitter and pull the "following" list for each user

In [1]:
import pandas as pd
import http.cookiejar 
import selenium.webdriver 
from selenium import webdriver 
from bs4 import BeautifulSoup as bs 
import time
import re
import csv

In [2]:
#Store twitter credentials
username =""
password =""

In [6]:
#Create broweser instance, sleep for 5 seconds to let webpage to load before trying to find_element
driver = webdriver.Chrome('chromedriver.exe')
driver.base_url = "https://twitter.com/login"
driver.get(driver.base_url)
time.sleep(5)

#Locate username/password fields and enter credentials
username_field = driver.find_element_by_name('session[username_or_email]')
password_field = driver.find_element_by_name('session[password]')
username_field.send_keys(username)
password_field.send_keys(password)

#Locate login button and click
driver.find_element_by_css_selector("[data-testid='LoginForm_Login_Button'][role='button']").click()

#Load list of users and store in data frame
data = pd.read_csv("test_users.csv")
allusers = data['Users']

results = []
nodes = [] 

#Loop through users and pull their following handles
for user in allusers:
    #Navigate to user profile and parse html
    driver.base_url = "https://twitter.com/"+user
    driver.get(driver.base_url)
    time.sleep(2)
    html_source = driver.page_source
    sourcedata = html_source.encode('utf-8')
    soup = bs(sourcedata)
   
    #Check if user is private, if so skip to next user
    if (soup.find('svg',{'aria-label':'Protected account'})):
        nodes.append([user,'private'])
        continue
    
    #Check if user is following 0, or more than 1000 people (potentially a bot), if so skip to next user
    following = soup.find('a',{'href':'/'+user+'/following'})['title']
    following = int(following.replace(',', ''))
    
    if (following == 0):
        nodes.append([user,following])
        continue
        
    if (following > 1000):
        nodes.append([user,following])
        continue
        
    nodes.append([user,following])
    
    #If user satisfies above conditions, navigate to 'following' page and pull 'following' list
    driver.base_url = "https://twitter.com/"+user+'/following'
    driver.get(driver.base_url)
    time.sleep(3)

    #Loop through page and pull content, scrolling until all users were found (counter == 0)
    counter = following
    
    while counter > 0:
        #Skip the scroll for first iteration, scroll for all other iterations 
        if (counter != following): 
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        html_source = driver.page_source
        sourcedata = html_source.encode('utf-8')
        soup = bs(sourcedata)

        #Find all profiles on webpage and store as separate lines
        frame = soup.find('div', {'aria-label':'Timeline: Following'})
        spans = frame.findAll('span',{'class':'css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0'})
        lines = [span.get_text() for span in spans]

        #Identify lines that contain user handle ("@"). Store in results[] if not there already, decrement counter
        for line in lines:
            at = re.search("^@.*",line)
            if(at):
                at = at.group(0)[1:]
                if([user,at] not in results):
                    results.append([user,at])
                    counter -= 1

#Add users, both source users and target users, to nodes list without duplicates
for i in results:
    if ([i[1],''] not in nodes):
        nodes.append([i[1],'']) 

#Write nodes user list to CSV
with open('Twitter_nodes.csv', 'w',newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['User','Following'])
    writer.writerows(nodes)
        
#Write results source/target list to CSV
with open('Twitter_edges.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Source','Target'])
    writer.writerows(results) 

print('Complete')

Complete
